In [1]:
import sys
import os
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [2]:
from python.fluorophores import FlStatic
from python.psfs import PsfDonut2D
from python.estimators import est_donut2d
from python.simulators import Simulator

In [3]:
fl = FlStatic()  # define a static fluorophore
fl.pos = [10, 0, 0]
fl.brightness = 1000 # kHz if excited at the center of a Gaussian beam

psf_donut = PsfDonut2D() # here you define a PSF. In this case, an analytical 2D donut PSF

sim = Simulator(fl)

numberOfLocalizations=1000

In [4]:
# define scan pattern
L = 75  # size of scan pattern
orbitpoints = 6  # number of probing points in orbit
probecenter = True  # should we also probe the center?
laserpower = 5  # relative, increases brightness
pointdwelltime = 0.1  # ms, measurement time in each point
repetitions = 2 # how often to repeat the pattern scan
sim.definePattern("donut", psf_donut, 
                  makepattern="orbitscan", 
                  orbitpoints=orbitpoints,
                  probecenter=probecenter, 
                  orbitL=L, 
                  pointdwelltime=pointdwelltime, 
                  laserpower=laserpower, 
                  repetitions=repetitions)

In [5]:
# we need an estimator. Define as component
fwhm=360  # size of the donut, needed for proper estimation. 
sim.defineComponent("estdonut", "estimator", est_donut2d, parameters=[sim.patterns["donut"].pos, L, fwhm], dim=(0,1))

In [6]:
# sequence: 
seq = ["donut", "estdonut"]
out = sim.runSequence(seq, maxlocs=numberOfLocalizations)
# args: maxlocs: number of times the pattern is scanned (one trace)
#       repetitions: number of times the sequence is repeated
# out.loc: localizations
# out.fluorophores: position of fluorophores
# out.raw: photon measurements

summary = sim.summarize_results(out)  # display summary of simulation

photch: 17.3, 25.8, 41.8, 49.4, 41.8, 25.7, 2.3, mean(phot): 204.2, signal phot: 204.2
std:  1.83, 1.60, 0.00, rmse: 2.28, 1.60, 0.00, pos: 8.63, -0.01, 0.00, bias: -1.37, -0.01, 0.00
locp: 1.86, 1.86, 0.00, sCRB: 1.83, 2.01, 0.00, sCRB*sqrt(phot): 26.1, 28.7, 0.0
